<a href="https://colab.research.google.com/github/marsggbo/AutoMLDemos/blob/master/ch4/nasbench101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 下载安装

In [ ]:
git clone https://github.com/google-research/nasbench
pip install ./nasbench

# 2. 下载查询数据库

- 下载 *nasbench_only108.tfrecord*: 提供的是对每个模型训练108 epochs后的实验数据。

`curl -O https://storage.googleapis.com/nasbench/nasbench_only108.tfrecord`

- 下载 *nasbench_full.tfrecord*: 提供的是完整的实验数据，包含了不同训练时长（4,12,36,108小时）的实验数据。

`curl -O https://storage.googleapis.com/nasbench/nasbench_full.tfrecord`

# 3. 使用NAS-Bench-101数据集

In [ ]:
# 1. 读取数据库

from nasbench import api
nasbench = api.NASBench('nasbench_only108.tfrecord') # 读取数据库可能会耗费较长时间，需耐心等待

In [ ]:
# 2. 预定义相关变量

# Useful constants
INPUT = 'input'
OUTPUT = 'output'
CONV3X3 = 'conv3x3-bn-relu'
CONV1X1 = 'conv1x1-bn-relu'
MAXPOOL3X3 = 'maxpool3x3'
NUM_VERTICES = 7
MAX_EDGES = 9
EDGE_SPOTS = NUM_VERTICES * (NUM_VERTICES - 1) / 2   # Upper triangular matrix
OP_SPOTS = NUM_VERTICES - 2   # Input/output vertices are fixed
ALLOWED_OPS = [CONV3X3, CONV1X1, MAXPOOL3X3]
ALLOWED_EDGES = [0, 1]   # Binary adjacency matrix


In [ ]:
# 3. 查询模型性能

# 从数据集中查询类似Inception的单元结构。
cell = api.ModelSpec(
  matrix=[[0, 1, 1, 1, 0, 1, 0],    # 输入层
          [0, 0, 0, 0, 0, 0, 1],    # 1x1卷积
          [0, 0, 0, 0, 0, 0, 1],    # 3x3卷积
          [0, 0, 0, 0, 1, 0, 0],    # 5x5卷积（由两个3x3卷积替代）
          [0, 0, 0, 0, 0, 0, 1],    # 5x5卷积（由两个3x3卷积替代）
          [0, 0, 0, 0, 0, 0, 1],    # 3x3最大池化
          [0, 0, 0, 0, 0, 0, 0]],   # 输出层
  
  ops=[INPUT, CONV1X1, CONV3X3, CONV3X3, CONV3X3, MAXPOOL3X3, OUTPUT] # 模块顶点的操作，与矩阵的顺序相匹配。
  )

# 多次查询可能会产生不同的结果。每个单元在每个训练轮次内被评估3次，并且查询将随机抽样其中一个结果。
data = nasbench.query(cell)
for k, v in data.items():
    print('%s: %s' % (k, str(v)))
  

输出结果会包括如下信息：
- 邻接矩阵（描述模块之间的连接关系）
- 模块的操作类型
- 可训练参数的数量
- 训练所用的时间（以秒为单位）
- 在训练集/验证集/测试集上的准确度。

例如
```python
module_adjacency: [[0 1 1 1 1 0 0]
 [0 0 0 0 0 0 1]
 [0 0 0 0 0 0 1]
 [0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0]
 [0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0]]
module_operations: ['input', 'conv3x3-bn-relu', 'conv1x1-bn-relu', 'maxpool3x3', 'conv3x3-bn-relu', 'conv3x3-bn-relu', 'output']
trainable_parameters: 2694282
training_time: 1155.85302734375
train_accuracy: 1.0
validation_accuracy: 0.9376001358032227
test_accuracy: 0.9311898946762085
```

